### Finding small molecule inhibitors for a COVID-19 Disease Map (DM) pathway

Choose the name of the COVID-19 DM pathway to target

In [1]:
pathway_name = 'Interferon 1 pathway'

Fetch the relevant pathway model from MMINERVA, and extract model elements

In [2]:
from covid_19.disease_maps.minerva_client import *
from covid_19.disease_maps.id_mapping_minerva import *

# Some basic elements of the COVID-19 DM
config = get_config(default_map_name)
project_id = get_project_id_from_config(config)
models = get_models(project_id, default_map_name)
ifn1_id = [m['idObject'] for m in models if m['name'] == pathway_name][0]
elements = get_model_elements(ifn1_id, project_id, default_map_name)

Now construct the INDRA mappings (i.e., equivalents using INDRA-compatible grounding namespaces and IDs) for these elements

In [3]:
indra_mappings = [indra_db_refs_from_minerva_refs(get_element_references(element))
                  for element in elements]

INFO: [2021-02-22 12:16:22] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/ben/.indra/bio_ontology/1.7/bio_ontology.pkl


In [4]:
indra_groundings = set(get_prioritized_db_refs_key(db_refs) for db_refs in indra_mappings if db_refs)

Filter down to all the human genes

In [5]:
human_genes = {entry for entry in indra_groundings if entry[0] == 'HGNC'}

In [12]:
print('We found a total of %d human genes in the %s model' % (len(human_genes), pathway_name)

SyntaxError: unexpected EOF while parsing (<ipython-input-12-9f83b0a2bbfc>, line 1)

We can now turn to the INDRA DB and find Inhibition statements targeting each of these human genes in the pathway of interest. We filter regulators to ones grounded to small molecule name spaces. We also filter out statements that have been curated as incorrect.

In [7]:
from indra.sources import indra_db_rest
from indra.tools import assemble_corpus as ac
try:
    from indra_db import get_db
    from indra_db.client.principal.curation import get_curations
    curs = get_curations(get_db('primary'))
except Exception:
    curs = []

In [8]:
inhibitors = {}
for db_ns, db_id in human_genes:
    ip = indra_db_rest.get_statements(object=f'{db_id}@{db_ns}', stmt_type='Inhibition',
                                      ev_limit=200)
    stmts = [s for s in ip.statements if set(s.subj.db_refs) & {'CHEBI', 'PUBCHEM', 'CAS', 'CHEMBL', 'DRUGBANK'}]
    stmts = ac.filter_by_curation(stmts, curations=curs)
    inhibitors[(db_ns, db_id)] = stmts

INFO: [2021-02-22 12:16:35] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=12036@HGNC&offset=0&type=Inhibition
INFO: [2021-02-22 12:16:35] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-02-22 12:16:39] indra.tools.assemble_corpus - Filtering 37 statements with any incorrect curations...
INFO: [2021-02-22 12:16:39] indra.tools.assemble_corpus - 37 statements after filter...
INFO: [2021-02-22 12:16:39] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=12033@HGNC&offset=0&type=Inhibition
INFO: [2021-02-22 12:16:39] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-02-22 12:16:42] indra.tools.assemble_corpus - Filtering 18 statements with any incorrect curations...
INFO: [2021-02-22 12:16:42] indra.tools.assemble_corpus - 18 statements after filter...
INFO: [2021-02-2

INFO: [2021-02-22 12:17:35] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=11362@HGNC&offset=1000&type=Inhibition
INFO: [2021-02-22 12:17:35] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-02-22 12:17:36] indra.tools.assemble_corpus - Filtering 178 statements with any incorrect curations...
INFO: [2021-02-22 12:17:36] indra.tools.assemble_corpus - 177 statements after filter...
INFO: [2021-02-22 12:17:36] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=5433@HGNC&offset=0&type=Inhibition
INFO: [2021-02-22 12:17:36] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-02-22 12:17:38] indra.tools.assemble_corpus - Filtering 6 statements with any incorrect curations...
INFO: [2021-02-22 12:17:38] indra.tools.assemble_corpus - 6 statements after filter...
INFO: [2021-02

INFO: [2021-02-22 12:18:41] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-02-22 12:18:42] indra.tools.assemble_corpus - Filtering 40 statements with any incorrect curations...
INFO: [2021-02-22 12:18:42] indra.tools.assemble_corpus - 40 statements after filter...
INFO: [2021-02-22 12:18:42] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=15631@HGNC&offset=0&type=Inhibition
INFO: [2021-02-22 12:18:42] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-02-22 12:18:45] indra.tools.assemble_corpus - Filtering 43 statements with any incorrect curations...
INFO: [2021-02-22 12:18:45] indra.tools.assemble_corpus - 39 statements after filter...
INFO: [2021-02-22 12:18:45] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=6131@HGNC&offset=0&type=Inhibition
INFO: [2021-02-22

INFO: [2021-02-22 12:19:28] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=3796@HGNC&offset=500&type=Inhibition
INFO: [2021-02-22 12:19:28] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-02-22 12:19:30] indra.tools.assemble_corpus - Filtering 213 statements with any incorrect curations...
INFO: [2021-02-22 12:19:30] indra.tools.assemble_corpus - 213 statements after filter...
INFO: [2021-02-22 12:19:30] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=7797@HGNC&offset=0&type=Inhibition
INFO: [2021-02-22 12:19:30] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-02-22 12:19:37] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=7797@HGNC&offset=500&type=Inhibition
INFO: [2021-02-22 12:19:37] indra.sources.indra_db_rest.ut

We can now check how many targets there are small molecule inhibitors for

In [9]:
has_inhibitors = len([gene for gene, inhs in inhibitors.items() if inhs])
print('We found inhibitors for %d of %d targets' % (len(inhibitors), has_inhibitors))

We found inhibitors for 46 of 44 targets


Now dump the results into a pickle, and a set of HTML files

In [10]:
import pickle
with open('ifn1_inhibitors.pkl', 'wb') as fh:
    pickle.dump(inhibitors, fh)

In [13]:
from indra.assemblers.html import HtmlAssembler
from indra.ontology.bio import bio_ontology
all_stmts = []
# Dump target-specific HTMLs
for gene_grounding, inhs in inhibitors.items():
    ha = HtmlAssembler(inhs, db_rest_url='https://db.indra.bio')
    ha.make_model()
    gene_name = bio_ontology.get_name(*gene_grounding)
    ha.save_model(f'{gene_name}.html')
    all_stmts += inhs
# Now dump for all inhibitors in a single file
ha = HtmlAssembler(all_stmts, db_rest_url='https://db.indra.bio')
ha.make_model()
pathway_code = pathway_name.replace(' ', '_').lower()
ha.save_model(f'all_inhibitors_{pathway_code}.html')

INFO: [2021-02-22 12:32:18] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:27363', '27363'}
INFO: [2021-02-22 12:32:18] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'63959', 'CHEBI:63959'}
INFO: [2021-02-22 12:32:18] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:27300', '27300'}
INFO: [2021-02-22 12:32:18] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'48706', 'CHEBI:48706'}
INFO: [2021-02-22 12:32:18] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:16243', '16243'}
INFO: [2021-02-22 12:32:18] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:64208', 'CHEBI:90693'}
INFO: [2021-02-22 12:32:18] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:24261', '24261'}
INFO: [2021-02-22 12:32:18] indra.a

INFO: [2021-02-22 12:32:23] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:25053', 'CHEBI:23053'}
INFO: [2021-02-22 12:32:23] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'16118', 'CHEBI:16118'}
INFO: [2021-02-22 12:32:23] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:64208', 'CHEBI:90693'}
INFO: [2021-02-22 12:32:23] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:64208', 'CHEBI:90693'}
INFO: [2021-02-22 12:32:23] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'87103', 'CHEBI:87103'}
INFO: [2021-02-22 12:32:24] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:27363', '27363'}
INFO: [2021-02-22 12:32:24] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'6021', '135398641'}
INFO: [2021-02-22 12:32: